# `CLOUD.jl` - 2D linear advection example

In [1]:
using CLOUD, OrdinaryDiffEq

┌ Info: Precompiling CLOUD [fb992021-99c7-4c2d-a14b-5e48ac4045b2]
└ @ Base loading.jl:1423


Define the physical problem

In [2]:
θ = π/4 # wave angle
a = sqrt(2) # wave speed
A = 1.0  # amplitude
L = 1.0  # domain length
k = (2*π/L, 2*π/L)  # wave number
T = 1.0/(a*max(abs(cos(θ)),abs(cos(θ)))) # end time for one period

conservation_law = LinearAdvectionEquation((a*cos(θ),a*sin(θ)))
initial_data = InitialDataSine(A,k)
exact_solution = ExactSolution(conservation_law,initial_data);

Set discretization parameters

In [3]:
M = 4
p = 5
p_map = 5
form = WeakConservationForm()
strategy = Lazy()

ode_algorithm = CarpenterKennedy2N54()
dt = 0.1*(L/M)/(a*(2*p+1));
write_interval = floor(Int, T/(dt*10));

## Collocated DGSEM on quadrilaterals

Set up a `DGSEM` scheme with LG quadrature on quadrilaterals of degree `p` with `M` edges in each direction

In [4]:
reference_approximation =ReferenceApproximation(
    DGSEM(p), Quad(),
    volume_quadrature_rule=LGQuadrature(),
    facet_quadrature_rule=LGQuadrature(),
    mapping_degree=p_map, N_plot=50)

mesh = warp_mesh(uniform_periodic_mesh(
    reference_approximation.reference_element, 
    ((0.0,L),(0.0,L)), (M,M)), 
    reference_approximation.reference_element, 0.2)

spatial_discretization = SpatialDiscretization(mesh, 
    reference_approximation)

results_path = save_project(conservation_law,
     spatial_discretization, initial_data, form, (0.0, T), Lazy(),
     "results/advection_2d_dgsem/", overwrite=true, clear=true)

"results/advection_2d_dgsem/"

Run the solver

In [5]:
ode_problem_1 = semidiscretize(conservation_law,
    spatial_discretization,
    initial_data, 
    form,
    (0.0, T),
    strategy)

save_solution(ode_problem_1.u0, 0.0, results_path, 0)

CLOUD_reset_timer()
sol = solve(ode_problem_1, ode_algorithm, adaptive=false, dt=dt,
    save_everystep=false, callback=save_callback(results_path, write_interval))
CLOUD_print_timer()

save_solution(last(sol.u), last(sol.t), results_path, "final")

 ────────────────────────────────────────────────────────────────────────────────
                                        Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:            423ms / 514.6%          57.3MiB / 2354.0%    

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 eval residual          49.9k    1.27s   58.4%  25.5μs    683MiB   50.6%  14.0KiB
   facet terms          49.9k    563ms   25.8%  11.3μs    260MiB   19.3%  5.34KiB
   volume terms         49.9k    480ms   22.0%  9.63μs    328MiB   24.3%  6.73KiB
   mass matrix solve    49.9k   90.0ms    4.1%  1.80μs   35.0MiB    2.6%     736B
 extrap solution        49.9k    576ms   26.4%  11.6μs    222MiB   16.4%  4.56KiB
 eval flux              49.9k    192ms    8.8%  3.84μs    109MiB    8.1%  2.24KiB
 eval num flux 

Calculate the integral L<sup>2</sup> error norm and conservation/energy error

In [6]:
error_analysis = ErrorAnalysis(results_path, conservation_law, 
    spatial_discretization)
conservation_analysis = PrimaryConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)
energy_analysis = EnergyConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)

println("L2 error:")
println(analyze(error_analysis, last(sol.u), initial_data, T)...)
println("Conservation (initial/final/diff):")
println(analyze(conservation_analysis)...)
println("Energy (initial/final/diff):")
println(analyze(energy_analysis)...)

L2 error:
0.0011999302847437096
Conservation (initial/final/diff):
[-1.3247403066879437e-8][-1.3247403068614161e-8][-1.734723475976807e-18]
Energy (initial/final/diff):
[0.12499988821253886][0.12494939330671882][-5.0494905820042124e-5]


## Modal DG scheme on triangles

Set up a `DGMulti` scheme on triangles of degree `p` with `M` edges in each direction

In [7]:
reference_approximation = ReferenceApproximation(
    DGMulti(p), Tri(), mapping_degree=p_map, N_plot=10)

mesh = warp_mesh(uniform_periodic_mesh(
    reference_approximation.reference_element, 
    ((0.0,L),(0.0,L)), (M,M)), 
    reference_approximation.reference_element, 0.1)

spatial_discretization = SpatialDiscretization(mesh, 
    reference_approximation)

results_path = save_project(conservation_law,
     spatial_discretization, initial_data, form, (0.0, T), Lazy(),
     "results/advection_2d_dgmulti/", overwrite=true, clear=true);

"results/advection_2d_dgmulti/"

Run the solver and print timer outputs

In [8]:
ode_problem_2 = semidiscretize(conservation_law,
    spatial_discretization,
    initial_data, 
    form,
    (0.0, T),
    strategy)

save_solution(ode_problem_2.u0, 0.0, results_path, 0)

CLOUD_reset_timer()
sol = solve(ode_problem_2, ode_algorithm, adaptive=false, dt=dt,
    save_everystep=false, callback=save_callback(results_path, write_interval))
CLOUD_print_timer()

save_solution(last(sol.u), last(sol.t), results_path, "final")

 ────────────────────────────────────────────────────────────────────────────────
                                        Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:          102μs / 4950040.4%      10.6KiB / 16794359.2%  

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 eval residual           100k    4.08s   81.2%  40.9μs    824MiB   47.6%  8.46KiB
   mass matrix solve     100k    3.24s   64.4%  32.4μs    396MiB   22.8%  4.06KiB
   volume terms          100k    485ms    9.6%  4.86μs    329MiB   19.0%  3.38KiB
   facet terms           100k    154ms    3.1%  1.54μs   31.4MiB    1.8%     330B
 eval flux               100k    340ms    6.8%  3.41μs    108MiB    6.2%  1.11KiB
 gather ext state        100k    226ms    4.5%  2.26μs    510MiB   29.4%  5.23KiB
 extrap solutio

Calculate the integral L<sup>2</sup> error norm and conservation/energy error

In [9]:
error_analysis = ErrorAnalysis(results_path, conservation_law, 
    spatial_discretization)
conservation_analysis = PrimaryConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)
energy_analysis = EnergyConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)

println("L2 error:")
println(analyze(error_analysis, last(sol.u), exact_solution, T)...)
println("Conservation (initial/final/diff):")
println(analyze(conservation_analysis)...)
println("Energy (initial/final/diff):")
println(analyze(energy_analysis)...)

L2 error:
0.00048784850808591414
Conservation (initial/final/diff):
[-6.5217300401244904e-9][-6.521728651044667e-9][1.3890798233884283e-15]
Energy (initial/final/diff):
[0.12499978051190094][0.12498968050064548][-1.0100011255467778e-5]
